In [1]:
import pandas as pd
from tqdm import tqdm
from glob import glob
import logging
import os
import re
from sklearn.model_selection import train_test_split

In [2]:
def get_lubm_graph_type(s):
    s = s.split('/')[-1]
    s = s.split('_')[-1]
    m = re.search("\d", s)
    return s[:m.start()]

In [3]:
def get_lubm_files_df(INPUT_GRAPHS_FOLDER, INFERENCE_GRAPHS_FOLDER):
    logging.info("Creating dataframe for LUBM1 input/inference pairs")
    rdf_files = []
    for input_graph_path in tqdm(sorted(glob(INPUT_GRAPHS_FOLDER + "*"))):
        input_graph_file = os.path.basename(input_graph_path)
        inference_path = INFERENCE_GRAPHS_FOLDER + input_graph_file
        graph_type = get_lubm_graph_type(input_graph_path)
        rdf_pair = {"input_graph_file": input_graph_path, "inference_file": inference_path, "graph_type": graph_type}
        rdf_files.append(rdf_pair)
    files_df = pd.DataFrame.from_dict(rdf_files)
    return files_df

In [4]:
lubm_files_df = get_lubm_files_df('datasets/graphs_with_descriptions/', 'datasets/jena_inference_with_descriptions/')

100%|████████████████████████████████████████████████████████████████████████| 17174/17174 [00:00<00:00, 100543.91it/s]


In [5]:
def train_validate_test_split(df, train_percent=0.6, validate_percent=0.2, stratify=None, seed=1):
    val_test_percent = 1 - train_percent
    test_percent = (1 - (train_percent + validate_percent))
    test_percent = test_percent / (test_percent + validate_percent)
    if stratify:
        df_train, df_val_test = train_test_split(df, test_size=val_test_percent, random_state=seed,
                                                 stratify=df[stratify])
        df_val, df_test = train_test_split(df_val_test, test_size=test_percent, random_state=seed,
                                           stratify=df_val_test[stratify])
    else:
        df_train, df_val_test = train_test_split(df, test_size=val_test_percent, random_state=seed)
        df_val, df_test = train_test_split(df_val_test, test_size=test_percent, random_state=seed)
    return df_train, df_val, df_test

In [6]:
rdf_data_train, rdf_data_val, rdf_data_test = train_validate_test_split(lubm_files_df,
                                                                        train_percent=0.6,
                                                                        validate_percent=0.2,
                                                                        stratify="graph_type",
                                                                        seed=1)

In [15]:
def write_nt_file(data, output_file):
    nt_files_orig = data['input_graph_file']
    nt_files_inferred = data['inference_file']
    nt_files = pd.concat([nt_files_orig, nt_files_inferred])

    with open(output_file, 'w') as outfile:
        for nt_file in nt_files:
            with open(nt_file, 'r') as infile:
                outfile.write(infile.read())
                outfile.write("\n")

In [16]:
write_nt_file(rdf_data_train, 'bin/lubm_train.owl')
write_nt_file(rdf_data_val, 'bin/lubm_val.owl')
write_nt_file(rdf_data_test, 'bin/lubm_test.owl')

In [17]:
with open('lubm.owl', 'w') as outfile:
    for nt_file in lubm_files_df['input_graph_file']:
        with open(nt_file, 'r') as infile:
            outfile.write(infile.read())
            outfile.write("\n")